## Example 

### Dataset Preparation

### Target Model Preparation

 #### either training from scratch

#### or loading pre-trained models
Pre-trained DDPM on FFHQ from [here](https://drive.google.com/file/d/1b69vT1dWzseXIFSz--2n8dsd_Zxiipu2/view?usp=drive_link)

### Membership Inference Attacks

#### Loss-based attack

In [ ]:
save_dir='results/ddpm_ffhq'
model_path='results/training_results/ddpm_ffhq/ddpm_ffhq.pth'
shuffled_idx_file='results/ffhq_1000_idx.npy'
data_path='results/ffhq_all.h5py'
diff_types='ddpm'

%run loss_attack.py \
--save_dir $save_dir \
--model_path $model_path \
--shuffled_idx_file $shuffled_idx_file \
--data_path $data_path \
--diff_types $diff_types

#### Likelihood-based attack

In [ ]:
save_dir='results/ddpm_ffhq'
model_path='results/training_results/ddpm_ffhq/ddpm_ffhq.pth'
shuffled_idx_file='results/ffhq_1000_idx.npy'
data_path='results/ffhq_all.h5py'
diff_types='ddpm'

%run likelihood_attack.py \
--save_dir $save_dir \
--model_path $model_path \
--shuffled_idx_file $shuffled_idx_file \
--data_path $data_path \
--diff_types $diff_types

### Show results
Take DDPM trained on FFHQ as an example.
After running loss_attack.py and likelihood_attack.py, we will get the result files `loss_ffhq_1000_ddpm.h5py` and `likelihood_ffhq_1000_ddpm.h5py`. Then, we summarize attack results as follows.

#### The loss-based attack against DDPM trained on FFHQ

##### TPR@ fixed FPR

In [ ]:
import h5py
import numpy as np


data_path='results/ddpm_ffhq/loss_ffhq_1000_ddpm.h5py'


with h5py.File(data_path, "r") as f:
    resluts = f['losses'][:]    # losses, smaller is better
print(resluts.shape)


# compute FPR@10%TPR, FPR@1%recall, FPR@0.1%recall, FPR@0.01%recall
from compute_metrics import get_metrics

labels = np.concatenate((np.zeros(1000), np.ones(1000))) 

all_steps_tpr_at_low_fpr=[]

print(resluts.shape[1])
for i in range(resluts.shape[1]):
    tpr_at_low_fpr_1 =get_metrics(labels, resluts[:,i], fixed_fpr=0.1,)
    tpr_at_low_fpr_2 =get_metrics(labels, resluts[:,i], fixed_fpr=0.01)
    tpr_at_low_fpr_3 =get_metrics(labels, resluts[:,i], fixed_fpr=0.001)
    tpr_at_low_fpr_4 =get_metrics(labels, resluts[:,i], fixed_fpr=0.0001)

    
    all_steps_tpr_at_low_fpr.append([tpr_at_low_fpr_1,tpr_at_low_fpr_2,tpr_at_low_fpr_3,tpr_at_low_fpr_4])

print('finished!')    
all_steps_tpr_at_low_fpr_ = np.reshape(all_steps_tpr_at_low_fpr,(-1,4))

In [ ]:
time_step0=0
time_step1=200
time_step2=500
time_step3=600
time_step4=800
time_step5=999

print('TPR@10%FPR,TPR@1%FPR,TPR@0.1%FPR,TPR@0.01%FPR')
print(all_steps_tpr_at_low_fpr_[time_step0]*100)
print(all_steps_tpr_at_low_fpr_[time_step1]*100)
print(all_steps_tpr_at_low_fpr_[time_step2]*100)
print(all_steps_tpr_at_low_fpr_[time_step3]*100)
print(all_steps_tpr_at_low_fpr_[time_step4]*100)
print(all_steps_tpr_at_low_fpr_[time_step5]*100)



##### TPR@ fixed FPR in all diffusion steps

In [ ]:
from compute_metrics import  plot_all_steps
save_path=None
plot_all_steps(all_steps_tpr_at_low_fpr_,save_path=save_path)

##### Log-scale ROC curve on certain diffusion steps

In [ ]:
from compute_metrics import  plot_one_step

time_step0=0
time_step1=200
time_step2=500
time_step3=600
time_step4=800
time_step5=999

labels = np.concatenate((np.zeros(1000), np.ones(1000))) 
predictions = [resluts[:,time_step0],resluts[:,time_step1],
              resluts[:,time_step2],resluts[:,time_step3],
              resluts[:,time_step4],resluts[:,time_step5]]

save_path=None
plot_one_step(labels, predictions,save_path)

#### The likelihood-based attack against DDPM trained on FFHQ

##### Log-scale ROC curve

In [ ]:
import h5py
import numpy as np



data_path='results/ddpm_ffhq/likelihood_ffhq_1000_ddpm.h5py'


with h5py.File(data_path, "r") as f:
    resluts = f['likelihood'][:]    # losses, smaller is better
print(resluts.shape)

labels = np.concatenate((np.zeros(1000), np.ones(1000))) 

all_steps_tpr_at_low_fpr=[]


from compute_metrics import get_metrics
tpr_at_low_fpr_1 =get_metrics(labels, resluts, fixed_fpr=0.1)
tpr_at_low_fpr_2 =get_metrics(labels, resluts, fixed_fpr=0.01)
tpr_at_low_fpr_3 =get_metrics(labels, resluts, fixed_fpr=0.001)
tpr_at_low_fpr_4 =get_metrics(labels, resluts, fixed_fpr=0.0001)


all_steps_tpr_at_low_fpr.append([tpr_at_low_fpr_1,tpr_at_low_fpr_2,tpr_at_low_fpr_3,tpr_at_low_fpr_4])

print('finished!')    
all_steps_tpr_at_low_fpr_ = np.reshape(all_steps_tpr_at_low_fpr,(-1,4))

In [ ]:
print('TPR@10%FPR,TPR@1%FPR,TPR@0.1%FPR,TPR@0.01%FPR')
print(all_steps_tpr_at_low_fpr_*100)

In [ ]:
def plot_TPR_at_low_FPR(labels, predictions,save_path=None):
    from sklearn.metrics import roc_curve
    import seaborn as sns
    sns.set_style("darkgrid")
    
    fpr_0, tpr_0, _ = roc_curve(labels, predictions)

    

    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(labels))]
    ns_fpr, ns_tpr, _ = roc_curve(labels, ns_probs)

    import matplotlib.pyplot as plt

    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='',c = 'grey')

    plt.plot(fpr_0, tpr_0, linewidth=2, label='Likelihood based attacks')

    
    plt.yscale('log')
    plt.xscale('log')
    plt.xlim([1e-5, 1])
    plt.ylim([1e-5, 1])
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    if save_path!=None:
        plt.savefig(save_path,bbox_inches = 'tight',pad_inches = 0)       
    # show the plot
    plt.show()


In [ ]:
save_path=None
plot_TPR_at_low_FPR(labels, resluts,save_path)